In [1]:
import pandas as pd
import seaborn as sns
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import GRU
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

In [3]:
#df = pd.read_csv("C:\dev\datasets\iot23_combined.csv")
df = pd.read_csv(r"C:\Users\Bas-Study\data\iot23_com\iot23_combined2.csv")

In [20]:
df

,Unnamed: 0,ts,id.orig_h,duration,orig_bytes,resp_bytes,missed_bytes,orig_pkts,orig_ip_bytes,resp_pkts,resp_ip_bytes,label,proto_icmp,proto_tcp,proto_udp,conn_state_OTH,conn_state_REJ,conn_state_RSTO,conn_state_RSTOS0,conn_state_RSTR,conn_state_RSTRH,conn_state_S0,conn_state_S1,conn_state_S2,conn_state_S3,conn_state_SF,conn_state_SH,conn_state_SHR
0,0,1.540470e+09,192.168.1.132,0.003497,0,0,0.0,5.0,212.0,3.0,144.0,Benign,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0
1,1,1.540470e+09,192.168.1.132,0.036724,34,311,0.0,1.0,62.0,1.0,339.0,Benign,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0
2,2,1.540469e+09,192.168.1.132,384.518261,15072,0,0.0,48.0,16416.0,0.0,0.0,Benign,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0
3,3,1.540470e+09,192.168.1.132,0.270332,48,48,0.0,1.0,76.0,1.0,76.0,Benign,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0
4,4,1.540470e+09,192.168.1.132,0.111429,48,48,0.0,1.0,76.0,1.0,76.0,Benign,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1446616,99994,1.532526e+09,192.168.100.111,0.000000,0,0,0.0,1.0,40.0,0.0,0.0,PartOfAHorizontalPortScan,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0
1446617,99995,1.532526e+09,192.168.100.111,0.000000,0,0,0.0,1.0,40.0,0.0,0.0,PartOfAHorizontalPortScan,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0
1446618,99996,1.532526e+09,192.168.100.111,0.000000,0,0,0.0,1.0,40.0,0.0,0.0,PartOfAHorizontalPortScan,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0
1446619,99997,1.532526e+09,192.168.100.111,0.000000,0,0,0.0,1.0,40.0,0.0,0.0,PartOfAHorizontalPortScan,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0


In [5]:
df['label'].value_counts()

PartOfAHorizontalPortScan     825939
Okiru                         262690
-   Benign   -                146275
DDoS                          138777
Benign                         53481
C&C                            15100
Attack                          3915
C&C-HeartBeat                    349
C&C-FileDownload                  43
C&C-Torii                         30
FileDownload                      13
C&C-HeartBeat-FileDownload         8
C&C-Mirai                          1
Name: label, dtype: int64

In [37]:
df['id.orig_h'].value_counts()

192.168.1.197      204393
192.168.1.198      199987
192.168.100.111    199935
192.168.1.195      123106
192.168.100.103    100647
                    ...  
87.140.41.9             1
221.135.134.4           1
103.5.159.28            1
69.108.2.131            1
200.170.196.189         1
Name: id.orig_h, Length: 3296, dtype: int64

In [6]:
df_node = df[df['id.orig_h']=='192.168.1.198']

In [7]:
df_node.shape

(199987, 28)

In [40]:
df_node['label'].value_counts()

Okiru                        112980
PartOfAHorizontalPortScan     56835
-   Benign   -                30016
C&C-HeartBeat                   114
C&C                              29
C&C-FileDownload                 13
Name: label, dtype: int64

In [8]:
pd.options.display.max_rows = 300
pd.options.display.max_columns = 300

In [9]:
df_node = df_node.drop(columns=['Unnamed: 0', 'id.orig_h'])

In [10]:
df_node['ts']

531573    1.545396e+09
531574    1.545396e+09
531575    1.545396e+09
531576    1.545396e+09
531577    1.545396e+09
              ...     
835988    1.547145e+09
835989    1.547145e+09
835990    1.547145e+09
835991    1.547145e+09
835992    1.547145e+09
Name: ts, Length: 199987, dtype: float64

In [11]:
x = df_node.loc[:, df_node.columns != 'label'].to_numpy()
df_y = df_node.loc[:, df_node.columns == 'label']
df_y = pd.get_dummies(df_y, columns=['label'])
y = df_y.to_numpy()

In [26]:
scaler = MinMaxScaler()

x_scaled = scaler.fit_transform(x)

In [12]:
df_node['label'].value_counts()

Okiru                        112980
PartOfAHorizontalPortScan     56835
-   Benign   -                30016
C&C-HeartBeat                   114
C&C                              29
C&C-FileDownload                 13
Name: label, dtype: int64

In [13]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = .2, random_state=42)

In [30]:
#6  labels
#24 features

model = Sequential()
model.add(Dense(24, input_dim=x.shape[1], activation='elu'))
model.add(Dense(18, activation='elu'))
model.add(Dense(12, activation='elu'))
model.add(Dense(y.shape[1], activation='sigmoid'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_23 (Dense)            (None, 24)                624       
                                                                 
 dense_24 (Dense)            (None, 18)                450       
                                                                 
 dense_25 (Dense)            (None, 12)                228       
                                                                 
 dense_26 (Dense)            (None, 6)                 78        
                                                                 
Total params: 1,380
Trainable params: 1,380
Non-trainable params: 0
_________________________________________________________________


In [31]:
model.fit(x_train, y_train, epochs=10, batch_size=10)

Epoch 1/10
15999/15999 [==============================] - 37s 2ms/step - loss: 555082.0000 - accuracy: 0.4235: 3s - 
Epoch 2/10
15999/15999 [==============================] - 34s 2ms/step - loss: 17679.0527 - accuracy: 0.5086
Epoch 3/10
15999/15999 [==============================] - 29s 2ms/step - loss: 0.9732 - accuracy: 0.5650
Epoch 4/10
15999/15999 [==============================] - 41s 3ms/step - loss: 0.9729 - accuracy: 0.5650
Epoch 5/10
15999/15999 [==============================] - 35s 2ms/step - loss: 0.9722 - accuracy: 0.5650
Epoch 6/10
15999/15999 [==============================] - 32s 2ms/step - loss: 0.9720 - accuracy: 0.5650
Epoch 7/10
15999/15999 [==============================] - 33s 2ms/step - loss: 0.9718 - accuracy: 0.5650
Epoch 8/10
15999/15999 [==============================] - 41s 3ms/step - loss: 0.9718 - accuracy: 0.5650
Epoch 9/10
15999/15999 [==============================] - 40s 2ms/step - loss: 0.9717 - accuracy: 0.5650
Epoch 10/10
15999/15999 [==============

In [32]:
preds = np.argmax(model.predict(x_test), axis = 1)
num_correct = 0

for i in range(0, len(preds)):
    if preds[i] == np.argmax(y_test[i]):
        num_correct+=1

num_correct/len(preds)

0.5646282314115706

In [64]:
model.save("/Users/kjwdamme/PythonProjects/datasets/first_model_scaled.h5")

In [43]:
preds.shape

(39998,)

In [37]:
np.unique(preds, return_counts=True)

(array([4], dtype=int64), array([39998], dtype=int64))

In [34]:
temp = np.transpose(y_test)


In [35]:
temp_df = pd.DataFrame(y_test)

In [36]:
temp_df[4].value_counts()

1    22584
0    17414
Name: 4, dtype: int64